In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler

In [33]:
df = pd.read_csv('../data/dataset_feature_engineering.csv')

### Limpieza de Datos
Comenzamos por convertir todas las variables categóricas a minúscula, para tener más uniforme nuestro dataset y evitar posibles errores por la escritura de algunas categorías. Luego transformamos las fechas de nacimiento y la fecha de la transacción en unix para que todas tengan un formato que podamos reconocer. Cabe recalcar que los datos del dataset no tenían valores nulos en ninguna de sus columnas y no se tenían datos en los campos numéricos.

In [ ]:
# Convertimos todas las letras a minúsculas
df['merchant'] = df['merchant'].str.lower()
df['category'] = df['category'].str.lower()
df['first'] = df['first'].str.lower()
df['last'] = df['last'].str.lower()
df['gender'] = df['gender'].str.lower()
df['state'] = df['state'].str.lower()
df['street'] = df['street'].str.lower()
df['city'] = df['city'].str.lower()
df['job'] = df['job'].str.lower()

In [ ]:
# Transformamos todo valor de fecha a datetime
df['dob'] = pd.to_datetime(df['dob'], format='%Y-%m-%d')
df['unix_time'] = pd.to_datetime(df['unix_time'], unit='s')

In [1]:
# Exportamos el dataset limpio
# df.to_csv('../data/dataset_cleaned.csv', index=False)

In [37]:
df.shape

(1852394, 35)

### Ingeniería de Características
Para la ingeniería de datos primero agrupamos y ordenamos por número de tarjeta.

In [38]:
# We sort the DataFrame by the card number and time
df = df.sort_values(['cc_num', 'unix_time'])
# We calculate the time difference in seconds between consecutive transactions per card
df['time_diff_seconds'] = df.groupby('cc_num')['unix_time'].diff().dt.total_seconds().fillna(0)
# This to know how often the client spends between transactions

# We calculate the hour window by unix time
df['hour_window'] = df['unix_time'].dt.floor('h')
# We calculate the amount of transactions per hour made per card
df['trans_per_hour'] = df.groupby(['cc_num', 'hour_window'])['trans_num'].transform('count')
# This to know how many transactions are made per hour by the client

# Ratio of transactions per hour to total transactions per card
df['hour_trans_ratio'] = df.groupby(['cc_num', 'hour'])['trans_num'].transform('count') / df.groupby('cc_num')['trans_num'].transform('count')
# This to know how many transactions the client usually makes per hour

In [ ]:
# We calculate if the distance between the client and the merchant is unusual (> 100 km)
df['unusual_distance'] = (df['dist_between_client_and_merch'] > 100).astype(int)
# This to know if the far transactions made by the client

# We calculate the distance between the client and the merchant
df['distance_diff'] = df.groupby('cc_num')['dist_between_client_and_merch'].diff().fillna(0)
# We calculate the velocity in km/h between consecutive transactions
df['velocity_km_h'] = (df['distance_diff'] / (df['time_diff_seconds'] / 3600)).replace([float('inf'), -float('inf')], 0).fillna(0)
# This to know how fast the client most move to make those transactions

# We calculate the mean and standard deviation of the distance between the client and the merchant
df['mean_dist_merchant'] = df.groupby('merchant')['dist_between_client_and_merch'].transform('mean')
df['std_dist_merchant'] = df.groupby('merchant')['dist_between_client_and_merch'].transform('std')
df['dist_z_score'] = (df['dist_between_client_and_merch'] - df['mean_dist_merchant']) / df['std_dist_merchant'].replace(0, 1)
# This to know how unusual the distance between the client and the merchant is

In [ ]:
# We calculate the amount ratio of transactions per month and year
df['amt_month_ratio'] = df['amt'] / df['amt_month'].replace(0, 1)
df['amt_year_ratio'] = df['amt'] / df['amt_year'].replace(0, 1)
# This helps us identify amounts that are significantly higher or lower than the average for that month or year.

# We calculate the mean and standard deviation of the amounts a client spent in a merchant
df['mean_amt_category'] = df.groupby('category')['amt'].transform('mean')
df['std_amt_category'] = df.groupby('category')['amt'].transform('std')
df['amt_z_score'] = (df['amt'] - df['mean_amt_category']) / df['std_amt_category'].replace(0, 1)
# This helps us identify transactions that are significantly higher or lower than the average for that category, which usually is the behavior of fraudulent transactions.

# We calculate the first time a client made a transaction with a high amount at a merchant
amt_threshold = df['amt'].quantile(0.9)
df['high_amt_first_time'] = (df['first_time_at_merchant'] & (df['amt'] > amt_threshold)).astype(int)
# This to know if the client made a high amount transaction for the first time at the merchant, which could mean a fraudulent transaction.

In [41]:
# We calculate the mean, standard deviation, and z-score of the amount of times spent at each merchant
df['mean_times_day_merchant'] = df.groupby('merchant')['times_shopped_at_merchant_day'].transform('mean')
df['std_times_day_merchant'] = df.groupby('merchant')['times_shopped_at_merchant_day'].transform('std')
df['times_day_z_score'] = (df['times_shopped_at_merchant_day'] - df['mean_times_day_merchant']) / df['std_times_day_merchant'].replace(0, 1)
# This helps us identify transactions that were made at the merchant that ocurred more frequently than usual, which usually is the behavior of fraudulent transactions.

In [42]:
# We calculate the amount of unique cards that were used at each merchant
df['unique_cards_per_hour'] = df.groupby(['merchant', 'hour_window'])['cc_num'].transform('nunique')
# This helps us identify if there is an usually high number of unique cards that were used at the merchant, which could indicate that a coordinated attack took place during a specific time.


# We calculate the variance of the amount spent in each hour window
df['amt_variance_hour'] = df.groupby(['merchant', 'hour_window'])['amt'].transform('std').fillna(0)
# This could helps us identify if there is a coordinated attack taking place by looking at the variance of the amount spent in each hour window. A very high amount could indicate that there are suspiciously high transactions being made and if the value is too low, it could indicate that there are many stolen cards that are being tested (which is used to check if a stolen card can actually make any purchase).

#### Mediciones y Variables Creadas:
- Indica qué tan seguido gasta el cliente entre transacciones
    - **time_diff_seconds**: Nos sirve para calcular la diferencia de tiempo entre transacciones
- Indica cuántas transacciones son hechas por hora por cliente
    - **hour_window**: Calcula una ventana de tiempo por hora.
    - **trans_per_hour**: Cantidad de transacciones por hora que se hace con la tarjeta.
- Nos indica la continuidad de las transacciones por hora por cliente
    - **hour_trans_ratio**: Ratio de la cantidad de transacciones totales hechas por hora.
- Nos ayuda a saber que tal lejos esta la transacción hecha por el usuario
    - **unusual_distance**: Registramos si la distancia entre la persona y el vendedor es mayor a 100 km.
- Indica que tan rápido se tuve que mover el cliente para realizar la transacción
    - **distance_diff**: Diferencia de distancia entre la persona y el vendedor.
    - **velocity_km_h**: Velocidad en la que se debería de mover la persona para poder realizar la compra después de la última transacción
- Indica si existe alguna distancia inusual entre el cliente y el vendedor
    - **mean_dist_merchant**: Distancia media entre la persona y el vendedor.
    - **std_dist_merchant**: Desviación estándar entre la persona y el vendedor.
    - **dist_z_score**: Cantidad de desviaciones estándar en la que se encuentra la distancia.
- Identifica si existen cantidades que son más grandes o pequeñas de lo usual acostumbrado por el cliente
    - **amt_month_ratio**: Ratio de la cantidad de transacciones hechas por mes.
    - **amt_year_ratio**: Ratio de la cantidad de transacciones hechas por año.
- Identifica si existen cantidad más altas o más bajas de las que compra el cliente por categoría
    - **mean_amt_category**: Cantidad media gastada por el usuario en una categoría específica de producto.
    - **std_amt_category**; Desviación estándar de la cantidad gastada por categoría por parte de la persona.
    - **amt_z_score**: Cantidad de desviaciones estándar donde se encuentra la cantidad gastada del mes por categoría.
- Identifica si el usuario hizo por primera vez una compra muy por encima del monto usual a un vendedor
    - **high_amt_first_time**: Registramos la primera vez que un usuario gasta una cantidad muy grande en un vendedor en específico.
- Nos ayuda a identificar si el usuario realizo transacciones más frecuentes de lo normal a un vendedor
    - **mean_times_day_merchant**: Cantidad de veces por día que la persona compra al vendedor.
    - **std_times_day_merchant**: Desviación estándar de la cantidad de veces que la persona compra con el vendedor.
    - **times_day_z_score**: Cantidad de desviaciones estándar en la que se encuentra la cantidad de compras realizadas por la persona al vendedor.
- Nos ayuda a identificar si un vendedor tiene una gran cantidad de tarjetas utilizadas en su comercio, inusualmente grande a lo normal.
    - **unique_cards_per_hour**: Cantidad de tarjetas diferentes que son utilizadas para comparar en el vendedor.
- Nos ayuda a identificar si sucede una cantidad inusual de compras dentro de una ventana de tiempo en específico.
    - **amt_variance_hour**: Variación en la cantidad gastada en una ventana de tiempo de una hora por parte de la persona
    - **is_coordinated_attack**: Variable que nos indica si la transacción es parte de un ataque coordinado, basándonos en una heurística personalizada.


In [43]:
df.shape

(1852394, 56)

#### Definición de la Variable Objetivo para Ataques Coordinados

Para determinar la variable is_coordinated_attack, inicialmente, se añade un indicador para marcar los ataques coordinados. Luego, se define ocho criterios individuales de sospecha basados en anomalías como el uso excesivo de tarjetas únicas en un corto periodo y en un mismo comerciante, velocidades de transacción físicamente imposibles, patrones de primera compra con montos inusualmente altos, distancias o frecuencias de compra extremas, varianza muy baja en los montos (sugiriendo pruebas de tarjetas) o intervalos de tiempo muy cortos entre transacciones de la misma tarjeta. Estos criterios individuales se combinan para formar cuatro patrones de ataque más complejos y robustos. Finalmente, una transacción se marca como parte de un ataque coordinado si, además de ser fraudulenta, coincide con al menos uno de estos patrones definidos, permitiendo así distinguir campañas de fraude orquestado dentro del conjunto de transacciones fraudulentas.

In [44]:
def identify_coordinated_fraud_attacks(df):
    """
    Identifica transacciones fraudulentas que provienen de ataques coordinados
    basándose en patrones específicos en las variables disponibles.
    """
    
    # Creamos copia del dataframe para no modificar el original
    df_analysis = df.copy()
    
    # Inicializamos el campo de ataque coordinado
    df_analysis['is_coordinated_attack'] = 0
    
    # Filtramos solo las transacciones que son fraudes
    fraud_mask = df_analysis['is_fraud'] == 1
    
    # CRITERIO 1: Múltiples tarjetas únicas en un periodo corto de tiempo
    # Indicador de bots o ataques automatizados
    high_unique_cards = df_analysis['unique_cards_per_hour'] > df_analysis['unique_cards_per_hour'].quantile(0.95)
    
    # CRITERIO 2: Velocidades imposibles entre transacciones
    # Indica uso de datos robados desde múltiples ubicaciones
    impossible_velocity = df_analysis['velocity_km_h'] > 200  # Más de 200 km/h es imposible
    
    # CRITERIO 3: Múltiples primeras transacciones en vendedores
    # Indica testing sistemático de tarjetas robadas
    first_time_pattern = df_analysis['first_time_at_merchant'] == 1
    
    # CRITERIO 4: Transacciones con cantidades inusuales pero similares entre sí
    # Patrón típico de ataques automatizados que prueban límites
    high_amt_first_time = df_analysis['high_amt_first_time'] == 1
    
    # CRITERIO 5: Anomalías extremas en distancia
    # Indica uso coordinado de datos desde ubicaciones dispersas
    extreme_distance_anomaly = (df_analysis['unusual_distance'] == 1) | \
                              (np.abs(df_analysis['dist_z_score']) > 2.5)
    
    # CRITERIO 6: Frecuencia anómala de transacciones por día a vendedores
    # Indica ataques sistemáticos a comercios específicos
    extreme_frequency_anomaly = np.abs(df_analysis['times_day_z_score']) > 4
    
    # CRITERIO 7: Varianza muy baja en cantidades por hora (patrones automatizados)
    # Los ataques coordinados tienden a usar cantidades muy pequeñas para probar que las tarjetas funcionan
    low_variance_amounts = df_analysis['amt_variance_hour'] < df_analysis['amt_variance_hour'].quantile(0.1)
    
    # CRITERIO 8: Patrones temporales sospechosos
    # Múltiples transacciones en ventanas de tiempo muy cortas
    short_time_diff = df_analysis['time_diff_seconds'] < 150  # Menos de 2.5 minutos entre transacciones
    
    # COMBINACIÓN DE CRITERIOS
    
    # Patrón 1: Ataque con múltiples tarjetas + velocidad imposible + primeras transacciones
    pattern_1 = high_unique_cards & (impossible_velocity | first_time_pattern)
    
    # Patrón 2: Velocidad imposible + distancia anómala + frecuencia extrema
    pattern_2 = impossible_velocity & extreme_distance_anomaly & extreme_frequency_anomaly
    
    # Patrón 3: Múltiples primeras transacciones + cantidades altas primera vez + baja varianza
    pattern_3 = first_time_pattern & high_amt_first_time & low_variance_amounts
    
    # Patrón 4: Combinación de múltiples anomalías (3 o más criterios)
    anomaly_count = (high_unique_cards.astype(int) + 
                    impossible_velocity.astype(int) + 
                    first_time_pattern.astype(int) + 
                    high_amt_first_time.astype(int) + 
                    extreme_distance_anomaly.astype(int) + 
                    extreme_frequency_anomaly.astype(int) + 
                    low_variance_amounts.astype(int) + 
                    short_time_diff.astype(int))
    
    pattern_4 = anomaly_count >= 3
    
    # Marcamos como ataque coordinado si cumple algún patrón Y es fraude
    coordinated_attack_mask = fraud_mask & (pattern_1 | pattern_2 | pattern_3 | pattern_4)
    
    df_analysis.loc[coordinated_attack_mask, 'is_coordinated_attack'] = 1
    
    return df_analysis['is_coordinated_attack']

# Aplicamos la función al dataset
df['is_coordinated_attack'] = identify_coordinated_fraud_attacks(df)

In [45]:
df[df['is_coordinated_attack'] == 1]

,cc_num,merchant,category,amt,first,last,gender,street,city,state,...,mean_amt_category,std_amt_category,amt_z_score,high_amt_first_time,mean_times_day_merchant,std_times_day_merchant,times_day_z_score,unique_cards_per_hour,amt_variance_hour,is_coordinated_attack
106628,60416207185,"fraud_windler, goodwin and kovacek",home,261.79,mary,diaz,f,9886 anita drive,fort washakie,wy,...,58.188180,48.504225,4.197610,1,1.773311,0.946483,0.239507,1,0.0,1
212993,60422928733,fraud_schmidt and sons,shopping_net,897.82,jeffrey,powers,m,38352 parrish road apt. 652,north augusta,sc,...,86.941974,244.277766,3.319492,1,1.626191,0.883611,-0.708673,1,0.0,1
133891,60423098130,fraud_beer-jast,kids_pets,5.21,jason,gray,m,875 amy point,amorita,ok,...,57.527851,48.729840,-1.073631,0,1.685575,0.870300,-0.787746,1,0.0,1
209419,60427851591,"fraud_moore, dibbert and koepp",misc_net,782.40,bradley,martinez,m,3426 david divide suite 717,burns flat,ok,...,80.181370,165.896953,4.232860,1,1.415135,0.711178,-0.583729,1,0.0,1
209536,60427851591,"fraud_reichert, rowe and mraz",shopping_net,1053.81,bradley,martinez,m,3426 david divide suite 717,burns flat,ok,...,86.941974,244.277766,3.958068,1,1.608953,0.828718,0.471870,1,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570487,4973530368125489546,"fraud_bradtke, torp and bahringer",personal_care,15.87,mary,rodriguez,f,8986 fitzgerald plains,winslow,ar,...,48.047896,49.284712,-0.652898,0,1.557377,0.822805,-0.677411,1,0.0,1
572346,4973530368125489546,"fraud_altenwerth, cartwright and koss",shopping_net,1018.47,mary,rodriguez,f,8986 fitzgerald plains,winslow,ar,...,86.941974,244.277766,3.813397,1,1.668603,0.928535,-0.720062,1,0.0,1
1132795,4980323467523543940,fraud_bashirian group,shopping_net,837.31,patrick,massey,m,7812 shane shoals apt. 607,north haverhill,nh,...,86.941974,244.277766,3.071782,1,1.629412,0.841295,-0.748146,1,0.0,1
1134646,4980323467523543940,fraud_flatley-durgan,misc_net,888.03,patrick,massey,m,7812 shane shoals apt. 607,north haverhill,nh,...,80.181370,165.896953,4.869581,1,1.432218,0.736070,-0.587197,1,0.0,1


In [46]:
# Export the dataset to a CSV file
df.to_csv('../data/data_engineered.csv', index=False)